In [1]:
import time
import pandas as pd
import numpy as np
data = pd.read_csv(r"C:\Users\86131\Desktop\毕设\data\movies.csv")
# 利用str可以对其中的每个元素进行处理，而无需循环
# 待解决：no genres listed
data = data.join(data.genres.str.get_dummies()).drop('genres',axis=1)
data.set_index(['movieId','title'], inplace=True)
data.head(5)

,,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,title,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
def _support(x,row):
    # axis=0，按行走，算的是每一列的和,=1案列走，算的是每一行的和
    ans = np.sum(x,axis=0) / row
    # reshape(-1),二维数组变一维
    return np.array(ans).reshape(-1)

def generate_new_combinations(old_combinations):
    # 二维数组扁平化后去重，得到所有项的类型
    items_types_in_previous_step = np.unique(old_combinations.flatten())
    for old_combination in old_combinations:
        max_combination = old_combination[-1]
        # mask是个True和False组成的数组
        mask = items_types_in_previous_step > max_combination
        valid_items = items_types_in_previous_step[mask]
        old_tuple = tuple(old_combination)
        for item in valid_items:
            yield from old_tuple
            yield item

# 参数：df：数据
# min_support：最小支持度
# use_colnames：是否使用列名,默认索引
def Apriori(df,min_support=0.5,use_colnames=False):
    value = df.values
    support = _support(value,value.shape[0])
    id = np.arange(value.shape[1])

    support_dict = {1: support[support >= min_support]}
    # reshape(-1,1)将一维数组转换为只有一列的二维数组,reshape参数（行，列），当为-1时，由总的/另一个参数确定
    itemset_dict = {1: id[support >= min_support].reshape(-1,1)}

    max_itemset = 1
    # all_ones 二维数组，列数为1
    all_ones = np.ones((value.shape[0],1))
    while max_itemset:
        next_max_itemset = max_itemset + 1

        # 由上方得到的频繁一项集生成候选二项集，对候选二项集处理得到频繁二项集，下一轮循环中由频繁二项集生成候选三项集
        # generate_new_combinations只是生成候选集，而不是频繁集
        combin = generate_new_combinations(itemset_dict[max_itemset])
        combin = np.fromiter(combin,dtype = int)
        combin = combin.reshape(-1,next_max_itemset)

        if combin.size == 0:
            break

        # 切片，[:, combin] 表示取所有行，但只取列索引为 combin 中指定的列。
        # 将同行中选中的列进行交运算
        # combin只说明了候选项集包含哪几列，要想计算频繁项集，还要将combin指定的列进行交集运算，只有三者都为一时_bools为True
        _bools = np.all(value[:, combin], axis=2)

        support = _support(np.array(_bools), value.shape[0])
        _mask = (support >= min_support).reshape(-1)
        print(_mask)
        if any(_mask):
            itemset_dict[next_max_itemset] = np.array(combin[_mask])
            support_dict[next_max_itemset] = np.array(support[_mask])
            max_itemset = next_max_itemset
        else:
            # Exit condition
            break

    all_res = []
    for k in sorted(itemset_dict):
        support = pd.Series(support_dict[k])
        itemsets = pd.Series([frozenset(i) for i in itemset_dict[k]], dtype="object")

        res = pd.concat((support, itemsets), axis=1)
        all_res.append(res)

    res_df = pd.concat(all_res)
    res_df.columns = ["support", "itemsets"]
    if use_colnames:
        mapping = {idx: item for idx, item in enumerate(df.columns)}
        res_df["itemsets"] = res_df["itemsets"].apply(
            lambda x: frozenset([mapping[i] for i in x])
        )
    res_df = res_df.reset_index(drop=True)

    return res_df

In [24]:
Apriori(data,min_support=0.05,use_colnames=True)

[ True False False False False  True False False False False False  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False False  True
 False False  True False False False False False  True False False False
  True False  True False False False False False False False False False
 False False False False False False]
[False False False False False False False False False False False False
 False]


,support,itemsets
0,0.187641,(Action)
1,0.129645,(Adventure)
2,0.062718,(Animation)
3,0.068158,(Children)
4,0.385547,(Comedy)
5,0.123075,(Crime)
6,0.447649,(Drama)
7,0.079963,(Fantasy)
8,0.100390,(Horror)
9,0.058817,(Mystery)
